# MNIST & AlexNet implementation
AlexNet是一個卷積神經網路，由亞歷克斯·克里澤夫斯基（Alex Krizhevsky）設計，與伊爾亞‧蘇茨克維（Ilya Sutskever）和克里澤夫斯基的博士導師傑弗里·辛頓共同發表，而辛頓最初抵制他的學生的想法。

AlexNet參加了2012年9月30日舉行的ImageNet大規模視覺辨識挑戰賽，達到最低的15.3%的Top-5錯誤率，比第二名低10.8個百分點。原論文的主要結論是，模型的深度對於提高效能至關重要，AlexNet的計算成本很高，但因在訓練過程中使用了圖形處理器（GPU）而使得計算具有可行性。

<center><img src="https://miro.medium.com/max/720/0*x-qsIInLP6pOMjVv" width="75%" height="75%"/></center>

In [1]:
import numpy as np
import torch
from torch import nn
from torchvision import datasets
from torchvision import transforms
from torch.utils.data import DataLoader
# change the max. print value
np.set_printoptions(threshold=1000000, suppress=True)

In [2]:
# 固定亂數種子
torch.manual_seed(1)
torch.cuda.manual_seed_all(1)

In [3]:
# 數據預處理
transform = transforms.Compose([
#     transforms.RandomResizedCrop(224),# 對圖像進行隨機裁減
#     transforms.RandomRotation(20),# 對圖像進行隨機旋轉
#     transforms.RandomHorizontalFlip(p=0.5),# 對圖像進行隨機水平翻轉
    transforms.Grayscale(num_output_channels=1),
    transforms.ToTensor()# 變成tensor格式
#     transforms.Normalize(
#         mean=[0.485, 0.456, 0.406],
#         std=[0.229, 0.224, 0.225])
]) # 數據增強

# 讀取數據
root = "./data/custom/MNIST"
train_data = datasets.ImageFolder(root + "/train",transform)
test_data = datasets.ImageFolder(root + "/test",transform)

In [4]:
batch_size = 256
train_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=test_data, batch_size=batch_size, shuffle=True)
feature_dataloader = DataLoader(dataset=train_data, batch_size=batch_size, shuffle=False)

In [5]:
classes = train_data.classes
classes_index = train_data.class_to_idx
print(classes)
print(classes_index)

['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
{'0': 0, '1': 1, '2': 2, '3': 3, '4': 4, '5': 5, '6': 6, '7': 7, '8': 8, '9': 9}


In [6]:
# 測試CUDA能否使用
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")

Using cuda device


In [7]:
# 轉換成CUDA能讀的格式
# generate data in parallel with PyTorch
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([256, 1, 28, 28])
Shape of y: torch.Size([256]) torch.int64


In [8]:
class AlexNet(nn.Module):   
    def __init__(self):
        super(AlexNet, self).__init__()
        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=5, stride=1, padding=1),
            nn.ReLU(inplace=True), 
            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),   
            nn.MaxPool2d( kernel_size=2, stride=2),
            nn.Conv2d(64, 96, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),                         
            nn.Conv2d(96, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),                         
            nn.Conv2d(64, 32, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d( kernel_size=2, stride=1),
        )
        self.classifier = nn.Sequential(
            nn.Dropout(),
            nn.Linear(32*12*12,2048),
            nn.ReLU(inplace=True),
            nn.Dropout(),
            nn.Linear(2048,1024),
            nn.ReLU(inplace=True),
            nn.Linear(1024,10),
        )
    
    def forward(self, x):

        x = self.features(x)
        x = x.view(x.size()[0],-1)
        x = self.classifier(x)
        return x

In [9]:
class FeatureExtractor(nn.Module):
    def __init__(self, model):
        super(FeatureExtractor, self).__init__()
        # Extract Net Feature Layers
        self.features = list(model.features)
        self.features = nn.Sequential(*self.features)
        # Convert the image into one-dimensional vector
        self.flatten = nn.Flatten()
        # Extract the first part of fully-connected layer from Net
        self.fc = model.classifier[1]
    def forward(self, x):
        # It will take the input 'x' until it returns the feature vector called 'out'
        out = self.features(x)
        out = self.flatten(out)
        out = self.fc(out) 
        return out 

# Initialize the model
model = AlexNet().to(device)
new_model = FeatureExtractor(model)

# Change the device to GPU
device = torch.device('cuda:0' if torch.cuda.is_available() else "cpu")
new_model = new_model.to(device)

In [10]:
model

AlexNet(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(96, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (classifier): Sequential(
    (0): Dropout(p=0.5, inplace=False)
    (1): Linear(in_features=4608, out_features=2048, bias=True)
    (2): ReLU(inplace=True)
    (3): Dropout(p=0.5, inplace=False)
    (4): Linear(in_features=2048, out_features=1024, bias=True)
    (5): ReLU(i

In [11]:
new_model

FeatureExtractor(
  (features): Sequential(
    (0): Conv2d(1, 32, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 96, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(96, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): Conv2d(64, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (10): ReLU(inplace=True)
    (11): MaxPool2d(kernel_size=2, stride=1, padding=0, dilation=1, ceil_mode=False)
  )
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc): Linear(in_features=4608, out_features=2048, bias=True)
)

In [12]:
features = []
step = 64
size = len(feature_dataloader.dataset)
model.train()
for batch, (X, y) in enumerate(feature_dataloader):
    X = X.to(device)# 讀圖片跟labels
    # Systematic Sampling
    sX = X[:len(X)-1:step]
    model.eval() # 測試模式，不做反向傳播
    with torch.no_grad(): # 不需要計算梯度
     # Compute prediction error
        feature = new_model(sX)
    if batch % 100 == 0: # 顯示進度條
        current = batch * len(X)
        print(f"Feature Extractor: [{current:>5d}/{size:>5d}]")   
    # Convert to NumPy Array, Reshape it, and save it to features variabl
    features.append(feature.cpu().detach().numpy())    

Feature Extractor: [    0/60000]
Feature Extractor: [25600/60000]
Feature Extractor: [51200/60000]


In [13]:
# Convert to NumPy Array
# 由於資料室不規則numpy因此會報錯，這裡因美觀我把它關掉
np.warnings.filterwarnings('ignore', category = np.VisibleDeprecationWarning)
featuresdata = np.array(features)

In [14]:
# 因為資料不平整，要用此方法去處理成一維numpy
import itertools
featureslist =list(itertools.chain(*featuresdata))

In [15]:
# 驗證輸出的資料量是否正確
# print(f"batch_size/step: {batch_size/step}",f"\n")
# print(f"batch[0]: ",len(featuresdata[0]))
print(f"Theoretical Value: {len(featuresdata[0]) * (len(featuresdata)-1) + len(featuresdata[234])}" )
print(f"Actual value: {len(featureslist)}")

Theoretical Value: 938
Actual value: 938


In [16]:
print("Theoretical out_features: 2048")
print(f"Actual out_features: {len(featureslist[0])}")

Theoretical out_features: 2048
Actual out_features: 2048


In [17]:
#...Rnnking 2048 * 938

In [18]:
lr = 1e-3
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [19]:
epochs = 10
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304650  [    0/60000]
loss: 0.067202  [25600/60000]
loss: 0.069229  [51200/60000]
Test Error: 
 Accuracy: 98.6%, Avg loss: 0.051074 

Epoch 2
-------------------------------
loss: 0.041422  [    0/60000]
loss: 0.027505  [25600/60000]
loss: 0.132680  [51200/60000]
Test Error: 
 Accuracy: 99.0%, Avg loss: 0.032668 

Epoch 3
-------------------------------
loss: 0.016410  [    0/60000]
loss: 0.066815  [25600/60000]
loss: 0.048849  [51200/60000]
Test Error: 
 Accuracy: 99.2%, Avg loss: 0.025989 

Epoch 4
-------------------------------
loss: 0.023878  [    0/60000]
loss: 0.021808  [25600/60000]
loss: 0.046209  [51200/60000]
Test Error: 
 Accuracy: 99.3%, Avg loss: 0.024488 

Epoch 5
-------------------------------
loss: 0.022801  [    0/60000]
loss: 0.006706  [25600/60000]
loss: 0.036250  [51200/60000]
Test Error: 
 Accuracy: 99.3%, Avg loss: 0.021750 

Epoch 6
-------------------------------
loss: 0.014515  [    0/60000]
loss: 0.034874  [256